In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
from pyspark.sql.functions import current_timestamp, concat, lit, col

In [0]:
%run "../includes/configuration"

In [0]:
name_schema = StructType(fields= [
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)
])

In [0]:
drivers_schema = StructType(fields= [
    StructField("driverId", IntegerType(), False),
    StructField("driverRef", StringType(), True),
    StructField("number", IntegerType(), True),
    StructField("code", StringType(), True),
    StructField("name", name_schema),
    StructField("dob", DateType(), True),
    StructField("nationality", StringType(), True),
    StructField("url", StringType(), True)
])

In [0]:
drivers_df = spark.read \
    .schema(drivers_schema) \
    .json(f"{raw_folder_path}/drivers.json")

In [0]:
drivers_df_renamed = drivers_df.withColumn("ingestion_date", current_timestamp()) \
    .withColumnRenamed("driverId", "driver_id") \
    .withColumnRenamed("driverRef", "driver_ref") \
    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))

In [0]:
drivers_final_df = drivers_df_renamed.drop(col("url"))

In [0]:
drivers_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.drivers")